# Fine-tuning GeneFormer using scarf library.

The scarf library enables fast handling of single-cell data. The helical library is a framework for training Foundation models from omics data.

Here we explore the interaction between scarf and helical. Will fine-tuning be faster, if we 

In [1]:
import helical

INFO:datasets:PyTorch version 2.5.1+cu118 available.
INFO:datasets:Polars version 1.21.0 available.
INFO:helical:Caduceus not available: If you want to use this model, ensure you have a CUDA GPU and have installed the optional helical[mamba-ssm] dependencies.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import umap
import pandas as pd
import numpy as np
import logging
import torch
from helical.utils import get_anndata_from_hf_dataset
from datasets import load_dataset

logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

# Import Geneformer & UCE from the Helical package
from helical import Geneformer,GeneformerConfig, UCE, UCEConfig
